년도, 월, 일, 시간, 교통량, 교통량측정지점명

In [1]:
from datetime import datetime
import pandas as pd

In [2]:
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql.types import *

conf = SparkConf()\
        .setAppName('spark-sql')\
        .set('spark.driver.extraClassPath', './data/mysql-connector-java-8.0.26.jar')

sc = SparkContext(conf=conf)

In [3]:
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [4]:
# MySQL로부터 테이블 호출
realtime_traffic = spark.read.format('jdbc')\
        .option('driver', 'com.mysql.jdbc.Driver')\
        .option('url', 'jdbc:mysql://{}:3306/{}'.format('3.37.159.174', 'finalPJT'))\
        .option('user', 'mulcam')\
        .option('password', 'mulcam')\
        .option('dbtable', 'realtime_traffic')\
        .load()

# spark에서 SQL 쿼리를 위해 테이블 선언
realtime_traffic.createOrReplaceTempView('realtime_traffic')

In [5]:
realtime_traffic = realtime_traffic.sort(['baseDate','baseTime'])

In [6]:
d = spark.sql("SELECT baseDate, baseTime, vol, spot_name FROM realtime_traffic ORDER BY baseDate DESC, baseTime DESC LIMIT 5")

In [7]:
d = d.select("*").toPandas()
d

,baseDate,baseTime,vol,spot_name
0,20211005,11,1212,종로(종로3가역)
1,20211005,11,1386,종로(동묘앞역)
2,20211005,11,1853,세종대로(시청역2)
3,20211005,11,930,대학로(한국방송통신대학교)
4,20211005,11,1554,율곡로(안국역)


In [8]:
d = d.astype({'vol':'float'})

In [9]:
# 교통량 빈값 평균값으로 처리
for idx, val in d.iterrows():
    if val['vol'] == 0:
        d['vol'][idx] = d['vol'].mean()

In [10]:
d

,baseDate,baseTime,vol,spot_name
0,20211005,11,1212.0,종로(종로3가역)
1,20211005,11,1386.0,종로(동묘앞역)
2,20211005,11,1853.0,세종대로(시청역2)
3,20211005,11,930.0,대학로(한국방송통신대학교)
4,20211005,11,1554.0,율곡로(안국역)


In [11]:
d['년도'] = ''
d['월'] = ''
d['일'] = ''

for idx, val in d.iterrows():
    d['년도'][idx] = d['baseDate'][idx][:4]
    d['월'][idx] = d['baseDate'][idx][4:6]
    d['일'][idx] = d['baseDate'][idx][6:]

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
d

,baseDate,baseTime,vol,spot_name,년도,월,일
0,20211005,11,1212.0,종로(종로3가역),2021,10,05
1,20211005,11,1386.0,종로(동묘앞역),2021,10,05
2,20211005,11,1853.0,세종대로(시청역2),2021,10,05
3,20211005,11,930.0,대학로(한국방송통신대학교),2021,10,05
4,20211005,11,1554.0,율곡로(안국역),2021,10,05


In [13]:
d['ds'] = d['년도'].apply(str)+'-' + d['월'].apply(str)+'-'  + d['일'].apply(str)+'-' + d['baseTime'].apply(str)
d['ds'] = pd.to_datetime(d['ds'],format='%Y-%m-%d-%H')

In [14]:
d

,baseDate,baseTime,vol,spot_name,년도,월,일,ds
0,20211005,11,1212.0,종로(종로3가역),2021,10,05,2021-10-05 11:00:00
1,20211005,11,1386.0,종로(동묘앞역),2021,10,05,2021-10-05 11:00:00
2,20211005,11,1853.0,세종대로(시청역2),2021,10,05,2021-10-05 11:00:00
3,20211005,11,930.0,대학로(한국방송통신대학교),2021,10,05,2021-10-05 11:00:00
4,20211005,11,1554.0,율곡로(안국역),2021,10,05,2021-10-05 11:00:00


In [15]:
accident_list = ['안국역사거리', '광하문우체국사거리','종로3가역사거리', '이화사거리', '동묘앞역사거리']
traffic_list = ['율곡로(안국역)', '세종대로(시청역2)', '종로(종로3가역)', '대학로(한국방송통신대학교)', '종로(동묘앞역)']

data = {}
for i in range(len(accident_list)):
    data[traffic_list[i]] = accident_list[i]

d['사고다발지점'] = ''
for idx, val in d.iterrows():
    d['사고다발지점'][idx] = data[val['spot_name']]

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [20]:
d = d.rename(columns={'vol':'y', 'baseTime':'시간', 'spot_name':'교통량측정지점명'})
d = d[['년도','월','일','시간','교통량측정지점명','사고다발지점','y','ds']]

In [21]:
d

,년도,월,일,시간,교통량측정지점명,사고다발지점,y,ds
0,2021,10,05,11,종로(종로3가역),종로3가역사거리,1212.0,2021-10-05 11:00:00
1,2021,10,05,11,종로(동묘앞역),동묘앞역사거리,1386.0,2021-10-05 11:00:00
2,2021,10,05,11,세종대로(시청역2),광하문우체국사거리,1853.0,2021-10-05 11:00:00
3,2021,10,05,11,대학로(한국방송통신대학교),이화사거리,930.0,2021-10-05 11:00:00
4,2021,10,05,11,율곡로(안국역),안국역사거리,1554.0,2021-10-05 11:00:00
